## Toxic: Word2Vec

In [1]:
import time
import pandas as pd

import numpy as np
import tensorflow as tf
import pandas as pd

import utils

checkpoint = "checkpoints/w2v_3allwords.ckpt"

/Users/brianmcmahon/anaconda3/envs/tensorflow1.4/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
df = pd.read_pickle('../data/toxictrain.pkl')
# df = df[df['rating']>0]
documents = [''.join(r) for r in df.comment_text]
text = ''.join(documents)

In [3]:
import re

text = re.sub('[^A-Za-z0-9 ]+', '',text)

In [4]:
words = utils.preprocess(text)
print(words[:30])

['i', 'hope', 'your', 'retarded', 'kids', 'get', 'anal', 'raped', 'and', 'murdered', 'for', 'having', 'such', 'a', 'fag', 'as', 'a', 'father', 'im', 'gonna', 'fuck', 'your', 'fat', 'wife', 'and', 'her', 'over', 'the', 'bridge', 'consider']


In [5]:
print("Total words: {}".format(len(words)))
print("Unique words: {}".format(len(set(words))))

Total words: 9910700
Unique words: 39064


In [6]:
vocab_to_int, int_to_vocab = utils.create_lookup_tables(words)
int_words = [vocab_to_int[word] for word in words]

In [7]:
from collections import Counter
import random

threshold = 1e-5
word_counts = Counter(int_words)
total_count = len(int_words)
freqs = {word: count/total_count for word, count in word_counts.items()}
p_drop = {word: 1 - np.sqrt(threshold/freqs[word]) for word in word_counts}
train_words = [word for word in int_words if random.random() < (1 - p_drop[word])]

In [8]:
def get_target(words, idx, window_size=5):
    ''' Get a list of words in a window around an index. '''
    
    R = np.random.randint(1, window_size+1)
    start = idx - R if (idx - R) > 0 else 0
    stop = idx + R
    target_words = set(words[start:idx] + words[idx+1:stop+1])
    
    return list(target_words)

In [9]:
def get_batches(words, batch_size, window_size=5):
    ''' Create a generator of word batches as a tuple (inputs, targets) '''
    
    n_batches = len(words)//batch_size
    
    # only full batches
    words = words[:n_batches*batch_size]
    
    for idx in range(0, len(words), batch_size):
        x, y = [], []
        batch = words[idx:idx+batch_size]
        for ii in range(len(batch)):
            batch_x = batch[ii]
            batch_y = get_target(batch, ii, window_size)
            y.extend(batch_y)
            x.extend([batch_x]*len(batch_y))
        yield x, y
    

In [10]:
train_graph = tf.Graph()
with train_graph.as_default():
    inputs = tf.placeholder(tf.int32, [None], name='inputs')
    labels = tf.placeholder(tf.int32, [None, None], name='labels')

In [11]:
n_vocab = len(int_to_vocab)
n_embedding = 200 # Number of embedding features 
with train_graph.as_default():
    embedding = tf.Variable(tf.random_uniform((n_vocab, n_embedding), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, inputs)

In [12]:
# Number of negative labels to sample
n_sampled = 100
with train_graph.as_default():
    softmax_w = tf.Variable(tf.truncated_normal((n_vocab, n_embedding), stddev=0.1))
    softmax_b = tf.Variable(tf.zeros(n_vocab))
    
    # Calculate the loss using negative sampling
    loss = tf.nn.sampled_softmax_loss(softmax_w, softmax_b, 
                                      labels, embed,
                                      n_sampled, n_vocab)
    
    cost = tf.reduce_mean(loss)
    optimizer = tf.train.AdamOptimizer().minimize(cost)

In [13]:
with train_graph.as_default():
    ## From Thushan Ganegedara's implementation
    valid_size = 16 # Random set of words to evaluate similarity on.
    valid_window = 100
    # pick 8 samples from (0,100) and (1000,1100) each ranges. lower id implies more frequent 
    valid_examples = np.array(random.sample(range(valid_window), valid_size//2))
    valid_examples = np.append(valid_examples, 
                               random.sample(range(1000,1000+valid_window), valid_size//2))

    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    # We use the cosine distance:
    norm = tf.sqrt(tf.reduce_sum(tf.square(embedding), 1, keep_dims=True))
    normalized_embedding = embedding / norm
    valid_embedding = tf.nn.embedding_lookup(normalized_embedding, valid_dataset)
    similarity = tf.matmul(valid_embedding, tf.transpose(normalized_embedding))

In [ ]:
# If the checkpoints directory doesn't exist:
# !mkdir checkpoints

In [ ]:
epochs = 50
batch_size = 1000
window_size = 10

with train_graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=train_graph) as sess:
    iteration = 1
    loss = 0
    sess.run(tf.global_variables_initializer())

    for e in range(1, epochs+1):
        batches = get_batches(train_words, batch_size, window_size)
        start = time.time()
        for x, y in batches:
            
            feed = {inputs: x,
                    labels: np.array(y)[:, None]}
            train_loss, _ = sess.run([cost, optimizer], feed_dict=feed)
            
            loss += train_loss
            
            if iteration % 100 == 0: 
                end = time.time()
                print("Epoch {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Avg. Training loss: {:.4f}".format(loss/100),
                      "{:.4f} sec/batch".format((end-start)/100))
                loss = 0
                start = time.time()
            
            if iteration % 1000 == 0:
                # note that this is expensive (~20% slowdown if computed every 500 steps)
                sim = similarity.eval()
                for i in range(valid_size):
                    valid_word = int_to_vocab[valid_examples[i]]
                    top_k = 8 # number of nearest neighbors
                    nearest = (-sim[i, :]).argsort()[1:top_k+1]
                    log = 'Nearest to %s:' % valid_word
                    for k in range(top_k):
                        close_word = int_to_vocab[nearest[k]]
                        log = '%s %s,' % (log, close_word)
                    print(log)
            
            iteration += 1
    save_path = saver.save(sess, "../data/checkpoints/w2v_3allwords.ckpt")
    embed_mat = sess.run(normalized_embedding)

Epoch 1/50 Iteration: 100 Avg. Training loss: 5.7186 0.3666 sec/batch
Epoch 1/50 Iteration: 200 Avg. Training loss: 5.6822 0.3798 sec/batch
Epoch 1/50 Iteration: 300 Avg. Training loss: 5.7193 0.4782 sec/batch
Epoch 1/50 Iteration: 400 Avg. Training loss: 5.6446 0.4700 sec/batch
Epoch 1/50 Iteration: 500 Avg. Training loss: 5.6233 0.4069 sec/batch
Epoch 1/50 Iteration: 600 Avg. Training loss: 5.6214 0.4122 sec/batch
Epoch 1/50 Iteration: 700 Avg. Training loss: 5.5528 0.3816 sec/batch
Epoch 1/50 Iteration: 800 Avg. Training loss: 5.4687 0.3896 sec/batch
Epoch 1/50 Iteration: 900 Avg. Training loss: 5.3900 0.3952 sec/batch
Epoch 1/50 Iteration: 1000 Avg. Training loss: 5.2638 0.3935 sec/batch
Nearest to time: astonished, youplease, decides, biggest, edit, complies, porch, copywritten,
Nearest to like: paras, punjabis, reportedly, frowned, opposing, fails, meet, hardcoded,
Nearest to your: themwhat, focuses, where, user, disagrees, facility, kof, clear,
Nearest to me: exaggeration, hawki

Epoch 2/50 Iteration: 4100 Avg. Training loss: 4.5747 0.3559 sec/batch
Epoch 2/50 Iteration: 4200 Avg. Training loss: 4.5665 0.3004 sec/batch
Epoch 3/50 Iteration: 4300 Avg. Training loss: 4.5516 0.0784 sec/batch
Epoch 3/50 Iteration: 4400 Avg. Training loss: 4.5420 0.2853 sec/batch
Epoch 3/50 Iteration: 4500 Avg. Training loss: 4.4922 0.3133 sec/batch
Epoch 3/50 Iteration: 4600 Avg. Training loss: 4.5144 0.3194 sec/batch
Epoch 3/50 Iteration: 4700 Avg. Training loss: 4.5119 0.3428 sec/batch
Epoch 3/50 Iteration: 4800 Avg. Training loss: 4.4807 0.3419 sec/batch
Epoch 3/50 Iteration: 4900 Avg. Training loss: 4.5081 0.2929 sec/batch
Epoch 3/50 Iteration: 5000 Avg. Training loss: 4.4909 0.2877 sec/batch
Nearest to time: porch, youplease, bedroom, hoaxing, astonished, complies, decides, alphabets,
Nearest to like: punjabis, reportedly, paras, vinnycee, 1884, existential, frowned, cosmology,
Nearest to your: automatically, themwhat, user, html, welcomehello, backgroundcolor, wikipedian, iar

Epoch 4/50 Iteration: 8100 Avg. Training loss: 4.3877 0.2612 sec/batch
Epoch 4/50 Iteration: 8200 Avg. Training loss: 4.4232 0.2545 sec/batch
Epoch 4/50 Iteration: 8300 Avg. Training loss: 4.3731 0.2477 sec/batch
Epoch 4/50 Iteration: 8400 Avg. Training loss: 4.3922 0.2454 sec/batch
Epoch 4/50 Iteration: 8500 Avg. Training loss: 4.3950 0.2490 sec/batch
Epoch 5/50 Iteration: 8600 Avg. Training loss: 4.1242 0.1224 sec/batch
Epoch 5/50 Iteration: 8700 Avg. Training loss: 4.2387 0.2229 sec/batch
Epoch 5/50 Iteration: 8800 Avg. Training loss: 4.3347 0.2360 sec/batch
Epoch 5/50 Iteration: 8900 Avg. Training loss: 4.3437 0.2749 sec/batch
Epoch 5/50 Iteration: 9000 Avg. Training loss: 4.3163 0.2727 sec/batch
Nearest to time: youplease, porch, bedroom, hoaxing, sudoghost, pilsudski, decides, copywritten,
Nearest to like: vinnycee, punjabis, existential, wishmanual, reportedly, extremly, 1884, frowned,
Nearest to your: automatically, themwhat, welcomehello, verticalaligntopcolor000000fontsize, e

Epoch 6/50 Iteration: 12100 Avg. Training loss: 4.2711 0.3437 sec/batch
Epoch 6/50 Iteration: 12200 Avg. Training loss: 4.2895 0.2952 sec/batch
Epoch 6/50 Iteration: 12300 Avg. Training loss: 4.2751 0.3052 sec/batch
Epoch 6/50 Iteration: 12400 Avg. Training loss: 4.2656 0.3148 sec/batch
Epoch 6/50 Iteration: 12500 Avg. Training loss: 4.2891 0.3246 sec/batch
Epoch 6/50 Iteration: 12600 Avg. Training loss: 4.2767 0.3145 sec/batch
Epoch 6/50 Iteration: 12700 Avg. Training loss: 4.2843 0.3169 sec/batch
Epoch 6/50 Iteration: 12800 Avg. Training loss: 4.2565 0.3780 sec/batch
Epoch 7/50 Iteration: 12900 Avg. Training loss: 3.8048 0.2198 sec/batch
Epoch 7/50 Iteration: 13000 Avg. Training loss: 4.1258 0.2471 sec/batch
Nearest to time: youplease, truculent, bedroom, porch, wong, sudoghost, scrubbing, vandals,
Nearest to like: vinnycee, punjabis, wishmanual, existential, roughshod, mumble, really, stay,
Nearest to your: everyone, themwhat, automatically, bradbury, welcomehello, truth, yopu, chea

In [ ]:
with train_graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=train_graph) as sess:
    saver.restore(sess, tf.train.latest_checkpoint('../data/checkpoints'))
    embed_mat = sess.run(embedding)

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [ ]:
viz_words = 500
tsne = TSNE()
embed_tsne = tsne.fit_transform(embed_mat[:viz_words, :])

In [ ]:
fig, ax = plt.subplots(figsize=(14, 14))
ax.axis([-200.0,200.0, -200.0,200.0])
ax = fig.gca()
ax.set_autoscale_on(False)

for idx in range(viz_words):
    plt.scatter(*embed_tsne[idx, :], color='steelblue')
    plt.annotate(int_to_vocab[idx], (embed_tsne[idx, 0], embed_tsne[idx, 1]), alpha=0.7)
plt.savefig('w2v_3_AllWords.png')